In [1]:
from pydantic import BaseModel, Field, ValidationError

class Model(BaseModel):
    number: int = Field(gt=0, lt=5)

In [2]:
from typing import Annotated

BoundedInt = Annotated[int, Field(gt=0, lt=5)]

class Model(BaseModel):
    number: BoundedInt

In [3]:
import pytz
from dateutil.parser import parse
from typing import Any
from datetime import datetime


def parse_datetime(v: Any) -> datetime:
        if isinstance(v, str):
            try:
                return parse(v)
            except Exception as e:
                raise ValueError(str(e))
        return v



def validate_date(dt: datetime) -> datetime:
        if dt.tzinfo is None:
            dt = pytz.utc.localize(dt)
        else:
            dt = dt.astimezone(pytz.utc)
        return dt

In [8]:
from pydantic import BeforeValidator, AfterValidator

DateTime = Annotated[datetime, BeforeValidator(parse_datetime)]

In [5]:
class Model(BaseModel):
    dt: DateTime

In [6]:
Model(dt="2020/1/1 3:00pm")

Model(dt=datetime.datetime(2020, 1, 1, 15, 0))

In [9]:
DateTime = Annotated[datetime, BeforeValidator(parse_datetime), AfterValidator(validate_date)]


In [10]:
class Model(BaseModel):
    dt: DateTime

In [11]:
Model(dt="2020/1/1 3:00pm")

Model(dt=datetime.datetime(2020, 1, 1, 15, 0, tzinfo=<UTC>))

In [20]:
def are_elements_unique(values: list[Any]):
    unique_elements = []  #not using set() because elements can be not hashable
    for value in values:
        if value in unique_elements:
            raise ValueError("elements must be unique")
        unique_elements.append(value)
    return unique_elements


In [21]:
UniqueIntegersList = Annotated[list[int], AfterValidator(are_elements_unique)]

In [22]:
class Model(BaseModel):
    numbers: UniqueIntegersList

In [24]:
m = Model(numbers=(1, 2, 3, 4, 5))
m

Model(numbers=[1, 2, 3, 4, 5])

In [25]:
try:
    Model(numbers=(1, 1, 2, 3, 4, 5))
except ValidationError as e:
    print(e)

1 validation error for Model
numbers
  Value error, elements must be unique [type=value_error, input_value=(1, 1, 2, 3, 4, 5), input_type=tuple]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error


In [27]:
from typing import TypeVar

T = TypeVar("T")

In [29]:
UniqueList = Annotated[list[T], AfterValidator(are_elements_unique)]


In [30]:
class Model(BaseModel):
    numbers: UniqueList[int] = []
    strings: UniqueList[str] = []

In [31]:
Model(numbers=(1, 2, 3, 4, 5), strings=["a", "b", "c"])

Model(numbers=[1, 2, 3, 4, 5], strings=['a', 'b', 'c'])

In [33]:
try:
    Model(numbers=["a", 2], strings=[1, "b"])
except ValidationError as e:
    print(e)

2 validation errors for Model
numbers.0
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='a', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/int_parsing
strings.0
  Input should be a valid string [type=string_type, input_value=1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type


In [41]:
UniqueList = Annotated[list[T], AfterValidator(are_elements_unique), Field(min_length=1, max_length=5, validate_default=True)]

class Model(BaseModel):
    numbers: UniqueList[int] = list
    strings: UniqueList[str] = list


In [37]:
try:
    Model(numbers=(1, 2, 3, 4, 6, 5, "&"), strings=[1, "b"])
except ValidationError as e:
    print(e)

2 validation errors for Model
numbers.6
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='&', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/int_parsing
strings.0
  Input should be a valid string [type=string_type, input_value=1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/string_type


In [42]:
Model()

ValidationError: 2 validation errors for Model
numbers
  Input should be a valid list [type=list_type, input_value=<class 'list'>, input_type=type]
    For further information visit https://errors.pydantic.dev/2.9/v/list_type
strings
  Input should be a valid list [type=list_type, input_value=<class 'list'>, input_type=type]
    For further information visit https://errors.pydantic.dev/2.9/v/list_type